In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [143]:
# Get Extensions
extension_url = 'https://addons.mozilla.org/en-US/firefox/extensions/'

In [693]:
req = requests.get(extension_url)

In [1299]:
master_soup = BeautifulSoup(req.content,'html.parser')

In [697]:
moz_link = 'https://addons.mozilla.org'

In [1319]:
for li in master_soup.select('.Categories-list')[0].select('li')[0]:
    print(li)
#     print(last_page_number(li.getText()))
# link = 
# moz_link+
# li.select('a')#[0]#.get('href')#+f'?page={x}'

<a class="Button Button--none Categories-link Categories--category-color-1" href="/en-US/firefox/extensions/category/accessibility/">Accessibility</a>


In [1445]:
# li in 
for x in master_soup.select('.Categories-list')[0].select('li')[:]:
    print(x.select('a')[0].get('href'))
# li

/en-US/firefox/extensions/category/accessibility/
/en-US/firefox/extensions/category/alerts-updates/
/en-US/firefox/extensions/category/appearance/
/en-US/firefox/extensions/category/bookmarks/
/en-US/firefox/extensions/category/download-management/
/en-US/firefox/extensions/category/feeds-news-blogging/
/en-US/firefox/extensions/category/games-entertainment/
/en-US/firefox/extensions/category/language-support/
/en-US/firefox/extensions/category/other/
/en-US/firefox/extensions/category/photos-music-videos/
/en-US/firefox/extensions/category/privacy-security/
/en-US/firefox/extensions/category/search-tools/
/en-US/firefox/extensions/category/shopping/
/en-US/firefox/extensions/category/social-communication/
/en-US/firefox/extensions/category/tabs/
/en-US/firefox/extensions/category/web-development/


In [1379]:

extension_dict = {}

for li in master_soup.select('.Categories-list')[0].select('li')[0]:
    for x in range(1,last_page_number(li.getText())):
        link = moz_link+li.select('a')[0].get('href')+f'?page={x}' # ]
        category = li.getText()
        soup = BeautifulSoup(requests.get(link).content)
        try:
            extension_dict[category].add(soup)
        except KeyError:
            extension_dict[category] = set()
            extension_dict[category].add(soup)            
for key, value in extension_dict.items():
    extension_dict[key] = list(extension_dict[key])

In [1391]:
extension_dict['Accessibility'][0]

<!DOCTYPE html>
<html dir="ltr" lang="en-US"><head><title data-react-helmet="true">Extensions in Accessibility – Add-ons for Firefox (en-US)</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link as="font" crossorigin="anonymous" href="/static-frontend/Inter-roman-subset-en_de_fr_ru_es_pt_pl_it.var.2ce5ad921c3602b1e5370b3c86033681.woff2" rel="preload" type="font/woff2"/><link crossorigin="anonymous" href="/static-frontend/amo-d8caa288c17e77f7b87c.css" integrity="sha512-CaIKHuNjRvVnd2ejiUmSKOp4uhVscw/ejFsvEZYb+zrWHYoC/KXg73hhoXkCgfo02rmAD1TOWU7tVKKLnLX52w==" rel="stylesheet" type="text/css"/><link href="/favicon.ico?v=2" rel="shortcut icon"/><link data-react-helmet="true" href="/en-US/firefox/opensearch.xml" rel="search" title="Firefox Add-ons" type="application/opensearchdescription+xml"/><link data-react-helmet="true" href="https://addons.mozilla.org/en-US/firefox/extensions/category/accessibility/" rel="canonical"/></head><body><div i

In [1385]:
alert_copy = extension_dict['Accessibility']

In [1387]:
for x in alert_copy[1:]:
    alert_copy[0].append(x)


In [1395]:
del_list = alert_copy[0].select('.SearchResult-wrapper')

In [1071]:

extension_requests = {}
for category,values in extension_dict.items():
    # print(extension_dict[category])#[]
    # print(len(values))
    print(values[0])

9


In [ ]:
for key,links in extension_dict.items():
    try:
        for link in extension_dict[key]:
            res = requests.get(link).content
            soup = BeautifulSoup(res,'html.parser')
            extension_requests[key].append(soup)
    except KeyError:
        extension_requests[key] = []
        res = requests.get(link).content
        soup = BeautifulSoup(res,'html.parser')
        extension_requests[key].append(soup)
        
extension_requests

In [1073]:
len(extension_dict['Accessibility'])

9

In [533]:
extension_soup = {}
for category, req in extension_requests.items():
    extension_soup[category] = BeautifulSoup(req,'html.parser')

In [639]:
import re

def replace_chars (value):
    pattern = r'[^a-zA-Z]+'
    return re.sub(pattern,'-',value).lower()

In [641]:
replace_chars('Feeds, News & Blogging')

'feeds-news-blogging'

In [931]:
# last_page = 
def last_page_number(category):
    return int(extension_soup[category].select('.Paginate-page-number')[0].getText().split()[-1]) + 1
# last_page

In [671]:
category

'Web Development'

In [683]:
last_page = last_page_number(category)
for x in range(1,last_page):
    next_url = f'https://addons.mozilla.org/en-US/firefox/extensions/category/{replace_chars(category.lower())}/?page={x}'
    req = requests.get(next_url)
    if req.status_code == 404:
        print(f'Page {x} not found')
        break
    else:
        print(f'Page {x} done!')
        continue
        
# except :

Page 1 done!
Page 2 done!
Page 3 done!
Page 4 done!
Page 5 done!
Page 6 done!
Page 7 done!
Page 8 done!
Page 9 done!
Page 10 done!
Page 11 done!
Page 12 done!
Page 13 done!


KeyboardInterrupt: 

In [651]:
extension_['Feeds, News & Blogging'].select('.SearchResult-wrapper')[-1]

<div class="SearchResult-wrapper"><div class="SearchResult-result"><div class="SearchResult-icon-wrapper"><img alt="Print to PDF" class="SearchResult-icon" src="https://addons.mozilla.org/user-media/addon_icons/844/844822-64.png?modified=1659762482"/></div><div class="SearchResult-contents"><h2 class="SearchResult-name"><a class="SearchResult-link" href="/en-US/firefox/addon/print-to-pdf-document/?utm_source=addons.mozilla.org&amp;utm_medium=referral&amp;utm_content=search">Print to PDF</a></h2><p class="SearchResult-summary">Easily print any page to PDF with just one click!</p><div class="SearchResult-metadata"><div class="SearchResult-rating"><div class="Rating Rating--small" title="Rated 3.5 out of 5"><div class="Rating-star Rating-rating-1 Rating-selected-star" title="Rated 3.5 out of 5"></div><div class="Rating-star Rating-rating-2 Rating-selected-star" title="Rated 3.5 out of 5"></div><div class="Rating-star Rating-rating-3 Rating-selected-star" title="Rated 3.5 out of 5"></div><

In [ ]:
extension_dict['Accessibility']

In [1097]:
lists = get_list('Accessibility')

# req = requests.get('https://addons.mozilla.org/en-US/firefox/extensions/category/accessibility/').content
req = requests.get('https://addons.mozilla.org/en-US/firefox/extensions/category/accessibility/?page=2000')

In [1117]:
sample_lists = extension_dict['Accessibility'][1].select('.SearchResult-wrapper')

In [1111]:
extension_dict['Accessibility'][1].append(extension_dict['Accessibility'][0])

In [1095]:
# soup.select('.AddonsCard-list')[0]
def get_list(category):
    for index,list in enumerate(extension_dict):
        order_list = extension_dict[category][index].select('.SearchResult-wrapper')
        append_soups = order_list
        return order_list

In [1419]:
for a in del_list:
    if int(a.select('span.SearchResult-users-text')[0].get_text().replace(',','').split()[0]) > 100:
        print(a.select('a.SearchResult-link')[0].get_text())
        print(a.select('span.SearchResult-users-text')[0].get_text())
        print('')

Dark Reader
1,158,716 users

Read Aloud: A Text to Speech Voice Reader
145,232 users

Sidebery
75,796 users

Dark Background and Light Text
39,146 users

Zoom Page WE
18,174 users

Dark Mode
29,081 users

Bionic Reader
3,511 users

Ai Chat everywhere
2,215 users

Mobile Dyslexic
2,110 users

Tab Mover
1,400 users

OpenDyslexic for Firefox
1,327 users

Symbaloo Homepage and Search
1,013 users

AInspector for WCAG Accessibility Evaluation
964 users

SuperStop
908 users

YourCodecs
797 users

Scroll Zoom
671 users

Fast Tab Switcher
617 users

pagenote
430 users

Read The Web Out Loud - RAO Text To Speech
242 users

Convert File
187 users



In [1431]:
new_df = {}
for a in del_list:
    try:    
        new_df['name'].append(a.select('a.SearchResult-link')[0].get_text())
        new_df['users'].append(a.select('span.SearchResult-users-text')[0].get_text())
        new_df['author'].append(a.select('h3.SearchResult-author')[0].get_text())
        new_df['summary'].append(a.select('.SearchResult-summary')[0].get_text())
        new_df['rating'].append(a.select('.SearchResult-rating')[0].get_text())
    except KeyError:
        new_df['name'] = []
        new_df['users'] = []
        new_df['author'] = []
        new_df['summary'] = []
        new_df['rating'] = []
        new_df['name'].append(a.select('a.SearchResult-link')[0].get_text())
        new_df['users'].append(a.select('span.SearchResult-users-text')[0].get_text())
        new_df['author'].append(a.select('h3.SearchResult-author')[0].get_text())
        new_df['summary'].append(a.select('.SearchResult-summary')[0].get_text())
        new_df['rating'].append(a.select('.SearchResult-rating')[0].get_text())
        
    
print('Done!')

Done!


In [1433]:
pd.DataFrame(new_df)

,name,users,author,summary,rating
0,Secure Password generator,1 user,Kunal Verma,Password generator extension allows you to eas...,There are no ratings yet
1,No Swipe - Disable Swipe,1 user,Suricata Tech,Disable trackpad two fingers navigation.,There are no ratings yet
2,Coordinates Copier for AP GIS Portal,1 user,ph4ni,Automatically creates latitude and longitude i...,There are no ratings yet
3,Pinterest True Fullscreen,1 user,LordFends,"When you fullscreen pinterest, the UI elements...",There are no ratings yet
4,Lords of War and Money Ext,1 user,Immortal Dynasty,Lords of War and Money - Firefox extension,There are no ratings yet
...,...,...,...,...,...
202,Lyrics Button,1 user,Valentino Amato,A button to rapidly search for the lyrics of a...,There are no ratings yet
203,Stopwatch Fun,1 user,Vn88,"StopWatch will have the Start, Stop, and Reset...",There are no ratings yet
204,ASN Plus,1 user,Shane,Aviation Safety Network Plus - Additional Feat...,There are no ratings yet
205,Essentialist: Scroll To Title,1 user,The Software Essentialist,"In The Software Essentialist web course, autom...",Rated 5 out of 5
